<a href="https://colab.research.google.com/github/LucasColas/2048-Clone/blob/main/IMDB_movie_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB movie reviews

This notebook contains the code to train a model from huggingface to analyze IMDB movie reviews.

Model : distilled bert.

Yay!
Enjoy!

## prerequisites

In [ ]:
!pip install --upgrade datasets fsspec s3fs huggingface_hub


INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 102.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling dataset

In [ ]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 118.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0


## libraries

In [ ]:
from collections import Counter
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import pipeline
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


## dataset

In [ ]:


# Load IMDB: train/test split with 25k examples each.
# Labels : 0 (negative), 1 (positive)
imdb = load_dataset("imdb")
train_ds, test_ds = imdb["train"], imdb["test"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
display(train_ds[0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
# Count labels in the training set
train_label_counts = Counter(train_ds["label"])
print("Label counts in the training dataset:", train_label_counts)

# Count labels in the testing set
test_label_counts = Counter(test_ds["label"])
print("Label counts in the testing dataset:", test_label_counts)

Label counts in the training dataset: Counter({0: 12500, 1: 12500})
Label counts in the testing dataset: Counter({0: 12500, 1: 12500})


In [ ]:
# merge half the test set with the training set.

test_ds_label_0 = test_ds.filter(lambda x: x['label'] == 0)
test_ds_label_1 = test_ds.filter(lambda x: x['label'] == 1)


num_to_move_0 = len(test_ds_label_0) // 2
num_to_move_1 = len(test_ds_label_1) // 2

# examples to move from test to train
move_ds_label_0 = test_ds_label_0.select(range(num_to_move_0))
move_ds_label_1 = test_ds_label_1.select(range(num_to_move_1))

# new training set
new_train_ds = concatenate_datasets([train_ds, move_ds_label_0, move_ds_label_1])

# the remaining examples are for the new test set
remain_ds_label_0 = test_ds_label_0.select(range(num_to_move_0, len(test_ds_label_0)))
remain_ds_label_1 = test_ds_label_1.select(range(num_to_move_1, len(test_ds_label_1)))
new_test_ds     = concatenate_datasets([remain_ds_label_0, remain_ds_label_1])

new_train_ds = new_train_ds.shuffle(seed=42)


train_ds, test_ds = new_train_ds, new_test_ds

# 9. Verify label distributions
print("Label counts in new TRAIN set:", Counter(train_ds["label"]))
print("Label counts in new TEST  set:", Counter(test_ds["label"]))

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Label counts in new TRAIN set: Counter({1: 18750, 0: 18750})
Label counts in new TEST  set: Counter({0: 6250, 1: 6250})


## model

In [ ]:


model_name = "distilbert-base-uncased"
tokenizer  = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"],
                     padding="max_length",
                     truncation=True,
                     max_length=256)

train_tok = train_ds.map(tokenize, batched=True)
test_tok  = test_ds.map(tokenize, batched=True)

# Set format for PyTorch
train_tok.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_tok.set_format ( "torch", columns=["input_ids", "attention_mask", "label"])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/37500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load pretrained model with a classification head
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Define training hyperparameters
args = TrainingArguments(
    output_dir="imdb-transformer",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds  = pred.predictions.argmax(-1)
    acc    = accuracy_score(labels, preds)
    p, r, f, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    compute_metrics=compute_metrics,
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## training

In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())


CUDA available: True


In [ ]:

trainer.train()
results = trainer.evaluate()
print(results)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lukyoda-13 (lukyoda-13-polytechnique-montr-al) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.281500,0.214610,0.914560,0.912843,0.916640,0.914737
2,0.174000,0.214928,0.912800,0.893173,0.937760,0.914924
3,0.115900,0.268148,0.910400,0.883637,0.945280,0.913420
4,0.079500,0.281438,0.915120,0.908004,0.923840,0.915854


{'eval_loss': 0.281438410282135, 'eval_accuracy': 0.91512, 'eval_precision': 0.9080044032080515, 'eval_recall': 0.92384, 'eval_f1': 0.9158537552541836, 'eval_runtime': 93.6073, 'eval_samples_per_second': 133.537, 'eval_steps_per_second': 4.177, 'epoch': 4.0}


In [ ]:
# Alternative : use fp16
args = TrainingArguments(
    output_dir="imdb-transformer",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    fp16_opt_level="O1",
    gradient_accumulation_steps=2,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    compute_metrics=compute_metrics,
)

trainer.train()
results = trainer.evaluate()
print(results)

In [ ]:

trainer.save_model("imdb-distilbert-finetuned")


# If your tokenizer isn’t in that folder yet, also do:
tokenizer.save_pretrained("imdb-distilbert-finetuned")


('imdb-distilbert-finetuned/tokenizer_config.json',
 'imdb-distilbert-finetuned/special_tokens_map.json',
 'imdb-distilbert-finetuned/vocab.txt',
 'imdb-distilbert-finetuned/added_tokens.json',
 'imdb-distilbert-finetuned/tokenizer.json')

## Inference

In [ ]:
# Load existing model
model   = AutoModelForSequenceClassification.from_pretrained("imdb-distilbert-finetuned")
tokenizer = AutoTokenizer.from_pretrained("imdb-distilbert-finetuned")

sentiment = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=0
)

Device set to use cuda:0


In [ ]:

nlp = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer, return_all_scores=True)
example = "This movie was not as good as I expected."
print(nlp(example))

Device set to use cuda:0


[[{'label': 'LABEL_0', 'score': 0.9944789409637451}, {'label': 'LABEL_1', 'score': 0.005521063692867756}]]


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
